In [1]:
import os
import pandas as pd
import numpy as np
from functools import reduce
from sklearn import clone
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm
from ML_util import *
from ML_Analysis_OptimalStimuli import LoadSelectiveData

path = "data/BlueMediumRarePupil_task1-1.csv"
thisData = LoadSelectiveData(path)

In [2]:
train_data_ratio = 0.6
valid_data_ratio = 0.2
test_data_ratio = 0.2

trb, vdb, teb = thisData.split_data(train_data_ratio, valid_data_ratio, test_data_ratio)

In [3]:
model = thisData.ml_train(trb)
confusionMatrixDict, jspiritSortedDict = thisData.ml_validate(model, vdb)

In [ ]:
results = thisData.ml_test_baseline(model, teb)

In [ ]:
confusionMatrixOptimal = results["cm_multiply"]
for i in range(13):
    print("====================")
    print(f"participant {i}")
    thisConfusionMatrix = confusionMatrixOptimal
    print(f"Accuracy for verification: {accuracyMeasurementForVerification(thisConfusionMatrix, i)}")
    print(f"FAR: {BiometricEvaluation(thisConfusionMatrix, i, 'FAR')}")
    print(f"FRR: {BiometricEvaluation(thisConfusionMatrix, i, 'FRR')}")
    tp = thisConfusionMatrix[i, i]
    fn = thisConfusionMatrix[i,:].sum() - tp
    fp = thisConfusionMatrix[:,i].sum() - tp
    print(f"tp: {tp}, fn: {fn}, fp: {fp}")
    

visualize_cm(confusionMatrixOptimal, 'RF', "Baseline")
# accuracyMeasurement(confusionMatrixOptimal)

In [ ]:
shape_cm = confusionMatrixDict['shape']
size_cm = confusionMatrixDict['size']
orientation_cm = confusionMatrixDict['orientation']
hue_cm = confusionMatrixDict['hue']
brightness_cm = confusionMatrixDict['brightness']

total_cm = reduce(lambda acc, cur: np.add(acc, cur), [shape_cm, size_cm, orientation_cm, hue_cm, brightness_cm], np.zeros((13,13)))

# np.save(os.path.join(os.getcwd(), 'ml-results', 'medium', 'new_task_cm', 'pupil_task1_val_shape_cm.npy'), shape_cm)
# np.save(os.path.join(os.getcwd(), 'ml-results', 'medium', 'new_task_cm', 'pupil_task1_val_size_cm.npy'), size_cm)
# np.save(os.path.join(os.getcwd(), 'ml-results', 'medium', 'new_task_cm', 'pupil_task1_val_brightness_cm.npy'), brightness_cm)
# np.save(os.path.join(os.getcwd(), 'ml-results', 'medium', 'new_task_cm', 'pupil_task1_val_hue_cm.npy'), hue_cm)
# np.save(os.path.join(os.getcwd(), 'ml-results', 'medium', 'new_task_cm', 'pupil_task1_val_orientation_cm.npy'), orientation_cm)
# np.save(os.path.join(os.getcwd(), 'ml-results', 'medium', 'new_task_cm', 'pupil_task1_val_total_cm.npy'), total_cm)

In [ ]:
visualize_cm(total_cm, 'RF', 'Val Total')
accuracyMeasurement(total_cm)

In [ ]:
jspiritSortedDict

In [4]:
strategy1 = thisData.stimuli_strategy1(jspiritSortedDict, teb)
results1 = thisData.ml_test_strategy(model, strategy1, teb)

SystemError: <built-in function asarray> returned NULL without setting an error

In [ ]:
for i, participant in enumerate(results1):
    print("====================")
    print(participant)
    thisConfusionMatrix = results1[participant]["cm_multiply"]
    print(f"Accuracy for verification: {accuracyMeasurementForVerification(thisConfusionMatrix, i)}")
    print(f"FAR: {BiometricEvaluation(thisConfusionMatrix, i, 'FAR')}")
    print(f"FRR: {BiometricEvaluation(thisConfusionMatrix, i, 'FRR')}")
    

In [ ]:
strategy2 = thisData.stimuli_strategy2(confusionMatrixDict, teb)
results2 = thisData.ml_test_strategy(model, strategy2, teb)

In [ ]:
targetParticipant = "participant_0"
targetConfusionMatrix = results2[targetParticipant]["cm_multiply"]
visualize_cm(normalize_cm(targetConfusionMatrix), 'rf', targetParticipant)


In [ ]:
for i, participant in enumerate(results2):
    # print("====================")
    # print(participant)
    thisConfusionMatrix = results2[participant]["cm_multiply"]
    # print(f"{accuracyMeasurementForVerification(thisConfusionMatrix, i)}    {BiometricEvaluation(thisConfusionMatrix, i, 'FAR')}    {BiometricEvaluation(thisConfusionMatrix, i, 'FRR')}")

    print(f"Accuracy for verification: {accuracyMeasurementForVerification(thisConfusionMatrix, i)}")
    print(f"FAR: {BiometricEvaluation(thisConfusionMatrix, i, 'FAR')}")
    print(f"FRR: {BiometricEvaluation(thisConfusionMatrix, i, 'FRR')}")

    tp = thisConfusionMatrix[i, i]
    fn = thisConfusionMatrix[i,:].sum() - tp
    fp = thisConfusionMatrix[:,i].sum() - tp
    print(f"tp: {tp}, fn: {fn}, fp: {fp}")

    visualize_cm(thisConfusionMatrix, 'rf', participant)

    

In [ ]:
strategy1 = thisData.stimuli_strategy1(jspiritSortedDict, teb)
thisData.ml_test_strategy_verification(model, strategy1, teb)